# Prompt 自动优化迭代（APO：Automatic Prompt Optimization）

在与大型语言模型进行交互时，有效的提示（Prompt）是至关重要的。提示可以理解为一种向模型提供输入的方式，它指导模型生成你期望的输出。而对于开发者来说，编写一个有效的提示可能是一项挑战。

在本教程中，我们将探讨如何使用我们的SDK来自动化这个过程，从而优化与大型模型的交互，同时减少对人工的要求。

APO 大致流程：

![](./awesome_demo/essay_scoring/img/apo.png)

SDK 内置了 APO 算法，能够直接快速使用该优化算法。具体来说，APO 通过利用大模型去评估初始 prompt 的好坏以及缺陷，然后继续利用大模型，根据反馈信息，不断调整 prompt 的内容，最终得到一个效果更好的 prompt。

为了使用 APO，我们需要指定一个初始的较为简单的用于描述任务的 prompt，如果可以的话还可以提供示例输入输出，用于给模型指定优化方向。

## 案例一：旅游攻略

这里我们以一个旅游攻略为例，来展示如何通过 APO 来提升任务效果。

首先我们先创建一个简单用于描述任务的 prompt

In [1]:
from qianfan.common import Prompt

p = Prompt("帮我制定一个在北海道旅游的攻略")

我们可以先使用该 prompt 测试下效果

In [2]:
from qianfan import Completion

comp = Completion(model="ERNIE-3.5-8K")

r = comp.do(prompt=p.render()[0])

print(r['result'])

[WARNING] [04-22 19:52:37] base.py:621 [t:140112452616448]: This key `messages` does not seem to be a parameter that the model `ERNIE-3.5-8K` will accept


当然，我很乐意帮助你制定一个北海道旅游的攻略。以下是一个基本的北海道行程规划，你可以根据自己的兴趣和需求进行调整。

第一天：札幌

抵达札幌后，首先可以去大通公园散步，欣赏周围的风景。然后，你可以去参观白色恋人工厂，了解北海道著名的甜点制作过程，并购买一些作为纪念品。晚上，你可以在狸小路商店街享受美食和购物。

第二天：小樽

从札幌出发，前往小樽。你可以漫步在小樽运河边，欣赏运河两岸的古老建筑和美丽的夜景。此外，你还可以参观八音盒馆和北一硝子，了解这些传统工艺的制作过程。最后，你可以登上天狗山，俯瞰小樽的全景。

第三天：洞爷湖

从小樽出发，前往洞爷湖。你可以乘船游览这个火山湖，欣赏湖光山色。此外，你还可以尝试一些火山温泉，享受一下泡温泉的乐趣。晚上，你可以在附近的羊蹄山欣赏美丽的星空。

第四天：旭川

前往旭川，参观著名的旭川动物园，欣赏各种可爱的动物。此外，你还可以参观旭川拉面博物馆，了解旭川拉面的历史和制作过程。

第五天：富良野

前往富良野，欣赏北海道美丽的自然风光。你可以在这里滑雪、泡温泉、欣赏花海等。此外，你还可以品尝到富良野的特产美食，如富良野牛肉等。

第六天：登别

前往登别，参观著名的登别地狱谷，感受一边是热气腾腾的温泉，一边是冰冷刺骨的河水的奇特景观。此外，你还可以品尝到登别的温泉蛋等美食。

第七天：回程

结束在北海道的旅程，返回札幌机场准备回程。

以上是一个基本的北海道7日游行程规划，你可以根据自己的兴趣和时间进行调整。在旅行过程中，记得要注意安全，保持好心情，享受这次美好的旅行。


从输出中可以发现输出较为宽泛，仅能用作参考而无法作为可实操的攻略，这很大程度是由于 prompt 过于简单导致的，但想要描述出更复杂的 prompt 却很难，因此此时我们可以使用 SDK 来自动对 prompt 进行优化。

In [4]:
optimized_p = p.apo_by_sample()
print(optimized_p.template)



我正在计划去北海道旅游，以下是我的具体需求和偏好：

* 旅行日期：从XXXX年XX月XX日至XXXX年XX月XX日，共计X天。
* 每日预算：XX元，包括住宿、交通、餐饮和门票等所有费用。
* 特别兴趣：我希望在北海道体验滑雪的乐趣，我的滑雪水平是（初学者/中级/高级），并品尝当地的美食。对于美食，我特别想尝试当地的传统料理和国际美食。

请基于以上信息，为我制定一个详细的旅游攻略，包括以下内容：

* 每天的详细行程安排，包括要参观的景点、要参加的活动和要品尝的美食。
* 推荐的滑雪场和餐厅，以及它们的特色和价格范围。
* 根据我的预算，合理地分配住宿、交通、餐饮和门票等费用。
* 攻略的格式和结构要清晰明了，包括明确的日期、时间段和活动内容，方便我阅读和参考。




可以看到优化后的 prompt 描述更为详细，增加了需要重点介绍的点，还通过 XXX 表示需要提供更多细节的地方。

接下来我们可以完善 prompt 描述后观察下效果

In [6]:
r = comp.do(prompt="""我正在计划去北海道旅游，以下是我的具体需求和偏好：

* 旅行日期：从2024年4月1日至2024年4月7日，共计7天。
* 每日预算：2000元，包括住宿、交通、餐饮和门票等所有费用。
* 特别兴趣：我希望在北海道体验滑雪的乐趣，我的滑雪水平是初学者，并品尝当地的美食。对于美食，我特别想尝试当地的传统料理和国际美食。

请基于以上信息，为我制定一个详细的旅游攻略，包括以下内容：

* 每天的详细行程安排，包括要参观的景点、要参加的活动和要品尝的美食。
* 推荐的滑雪场和餐厅，以及它们的特色和价格范围。
* 根据我的预算，合理地分配住宿、交通、餐饮和门票等费用。
* 攻略的格式和结构要清晰明了，包括明确的日期、时间段和活动内容，方便我阅读和参考。""")

print(r['result'])

[WARNING] [04-22 20:00:44] base.py:621 [t:140112452616448]: This key `messages` does not seem to be a parameter that the model `ERNIE-3.5-8K` will accept


北海道旅游攻略

一、行程概览

日期：2024年4月1日至2024年4月7日，共7天
预算：每日2000元
主题：滑雪体验、品尝美食

二、行程安排

Day 1: 抵达札幌

* 上午：抵达新千岁机场，乘坐巴士或地铁前往札幌市区。
* 下午：入住札幌市区酒店（预算约500元）。
* 晚上：在札幌市区品尝当地美食，如北海道拉面、毛蟹等（预算约300元）。

Day 2: 札幌市区游

* 上午：参观大通公园，体验札幌市民的日常生活。
* 下午：前往白色恋人工厂，了解北海道著名的甜点制作过程，并品尝其他美食（门票+购物预算约200元）。
* 晚上：在狸小路商店街闲逛，品尝各种小吃（预算约200元）。

Day 3: 札幌滑雪初体验

* 上午：前往札幌市郊的滑雪场（如札幌国际滑雪场），体验滑雪乐趣（滑雪费用+交通预算约400元）。
* 下午：继续在滑雪场练习滑雪技能。
* 晚上：返回酒店休息。

Day 4: 前往小樽

* 上午：乘坐巴士或火车前往小樽（交通预算约200元）。
* 下午：漫步在小樽运河，感受《情书》中的浪漫氛围。
* 晚上：品尝小樽海鲜料理（预算约300元）。

Day 5: 小樽深度游

* 上午：参观八音盒馆、北一硝子等著名景点。
* 下午：前往天狗山，俯瞰小樽全景。
* 晚上：在小樽市区品尝当地美食（预算约200元）。

Day 6: 滑雪进阶体验

* 上午：返回札幌，继续前往滑雪场进行滑雪进阶体验（滑雪费用+交通预算约400元）。
* 下午：在札幌市区闲逛，购买纪念品。
* 晚上：品尝札幌国际美食（预算约300元）。

Day 7: 离开札幌

* 上午：退房，前往新千岁机场。
* 下午：乘坐飞机返回家乡。

三、费用预算

住宿：7天 x 500元/天 = 3500元
交通：札幌-小樽往返 x 200元/次 = 400元
       滑雪场交通 x 100元/次 = 400元
       其他交通费用（地铁、巴士等）约200元
       总计：1000元
餐饮：7天 x 300元/天 = 2100元
滑雪：2次 x 200元/次 = 400元
门票：白色恋人工厂、八音盒馆等约200元
其他：购物、纪念品等预留300元
总计：7500元

四、注意事项

1. 根据个人兴趣和体力状况，适当调整行程安排。
2. 滑雪时注意安全，遵守

可以看到相较于之前的输出，给出的结果更加丰富，为每天都做出了更为详细的安排，而且增加了预算和注意事项，能够更好的完成我们的任务。

## 案例二：作文批改

接下来我们以一个更为复杂的作文批改任务为例，在该案例中，prompt将具有参数，我们可以通过提供示例输入输出进行优化。

在开始时，我们需要首先准备一个初始prompt，以及示例输入输出。

In [28]:
prompt = Prompt("请帮我批改如下高考作文，从内容、表达、发展等级、扣分项和残篇评定几个角度给出评价和评分（0-60分）：\n题目：{question}\n作文内容：{article}")

target = {
    "question": "人们因技术发展得以更好地掌控时间，但也有人因此成了时间的仆人。这句话引发了你怎样的联想与思考？请写一篇文章。要求：选准角度，确定立意，明确文体，自拟标题；不要套作，不得抄袭；不得泄露个人信息；不少于800字。",
    "article": """
    我的世界大于这世界
    我们消磨时间，也悔恨虚度了时间。在新时代的技术潮流中，当我们学会了用充满科技感的软件感受大千世界，便开始用无限的信息充斥着所有时间——浏览新鲜事儿，刷刷直播视频，关注好友动态……直到耗尽睡前的全部精力。当我们想要获得更多，便开始想尽办法“抢先”“争先”，不断被向前驱赶。
    可是，我们的世界不止有这些，我们还可以去欣赏“江间波浪兼天涌，塞上风云接地阴”的壮阔，还可以去感受“揉碎在浮躁间，沉淀着彩虹似的梦”的温柔，更可以来一场“携手藕花湖上路，一霎黄梅细雨”的闲庭信步。只有当我们做了时间的主人才更能体会——我的世界，大于这世界。
    时不重至，华不再阳。只有做时间的朋友，才能拥有有意义的人生，创造更有价值的事业。55岁的翟志刚，一直都和所有航天员一样，进行着日复一日的训练。13年间，他先后成为神舟十号和神舟十二号任务的备份航天员。“把结果交给时间，不虚度时间”，让他把梦想放飞在了太空。在北京冬奥会上，谷爱凌流畅的滑线感动了世界，这名18岁的姑娘在接受多家媒体采访时，都曾讲述过在车上写作业，甚至在训练和学业同样繁忙时仍能保持很长睡眠时间的故事。“珍惜时间，合理规划时间”，让她用热爱把冰雪融化。
    现如今，技术的进步，正在提高人们利用时间的效率，互联网拉进了人们彼此之间的距离。通过聊天软件，我们可以随时获取信息和交流。过去，一封信是车马慢慢，递送到亲人的手里；现在，邮件秒达在同事的邮箱；功能丰富的购物平台，让买卖的时间大大缩短，人们不再需要通过“逛街”耗费时间和精力；于我们学生来说，在线教育，使得我们可以在自己的时间和节奏下，学习新的技能和知识……通过技术省力省时的现象，在我们身边不胜枚举。
    然而，技术是把双刃剑，它让我们对时间的控制越来越精准、自信，同时，也让我们逐渐模糊了时间的概念，我们开始漫无目的地趴在“网”上，消耗时光。再如“节点”“抢先”，也让我们对时间的把控失了味儿。可如果只为达到“节点”，就不会拥有太空漫步的本领，最后只能与梦想擦肩；假如只为一时“抢先”，就不会有一手提着雪板，一手拿着食物走入赛场的从容。我们可以拥抱技术，用时间工具更好地为我们服务，让技术为生活增香添色。但是，我们不要让“科技”、让“时限”，无孔不入包围我们的生活，失去张弛有度的人生。
    总之，在通过技术精确利用时间的同时，我们不要愈发深入地陷在时间的束缚和催促下。只有汲取科技的力量，善用时间，才能感受生活的美好，延展自我的世界。正如杜牧所言：“浮生若梦冰如水，日夜东流人不知。”韶华易逝，我们能做的就是在有限的生命长度里拓宽自己无限的宽度和深度。这样，我的世界便大于这世界。
    """,
    "expect": """
详细解析：
    内容项: 
        解析: 文章围绕技术发展对人们时间观念的影响进行了深入的思考和讨论，立意明确，中心突出，内容充实，思想健康，感情真挚。作者通过列举翟志刚和谷爱凌的事例，以及现代科技带来的便利和负面影响，阐述了做时间主人的重要性，并提出了在利用科技的同时，也要善用时间，拓宽生命宽度的观点。整篇文章逻辑清晰，论述有力，符合题意。,
        等级: 一等
        得分: 20分
    
    表达项: 
        解析: 文章采用议论文的文体，结构严谨，语言流畅，字迹工整，符合文体要求。作者在论述过程中运用了多种论证方法，如举例论证、对比论证等，使文章更具说服力。同时，作者还引用了一些诗句和名言，增加了文章的文采和深度。,
        等级: 一等
        得分: 20分
    
    发展等级:
        解析: 文章在立意、内容和表达等方面都表现出较高的水平，尤其是在思想深度和文采方面，作者通过深入的思考和精心的表达，使文章具有了一定的创意和启发性。因此，本文在发展等级方面也可以给予较高的评价。,
        等级: 一等
        得分: 20分
    
    扣分项和残篇评定: 
        解析: 文章在字数、标题、错别字、标点符号等方面都没有明显的问题，因此不需要扣分。同时，文章也不是残篇，结构完整，论述充分。,
        扣分: 0分
    

缺点和改进意见: 
    缺点: 虽然文章整体表现优秀，但在一些细节方面还可以进一步完善。例如，部分论据的阐述可以更加具体和生动，以增加读者的阅读兴趣和理解深度；另外，个别句子的表达可以更加精炼和准确，以提高文章的整体质量。,
    改进意见: 作者可以在今后的写作中注重论据的具体性和生动性，通过描绘更多的细节和场景来增强文章的说服力和感染力；同时，也可以加强对句子表达的锤炼和打磨，力求用更精炼、更准确的语言来表达自己的思想和观点。

最终得分: 60分
    """
}

我们可以先来看看初始 prompt 的输出结果

In [29]:
r = comp.do(prompt=prompt.render(**target)[0])

print(r['result'])

[WARNING] [04-23 11:59:12] base.py:621 [t:140112452616448]: This key `messages` does not seem to be a parameter that the model `ERNIE-3.5-8K` will accept


这篇作文是一篇论述技术发展对时间利用影响的文章，作者从个人经历和社会现象出发，深入探讨了技术发展对时间掌控的影响，以及如何在技术的影响下更好地利用时间。

在内容方面，作者首先描述了人们在新时代技术潮流中如何消磨时间，接着提出了自己的观点，即我们应该做时间的主人，去欣赏生活的美好。然后，作者通过举例航天员翟志刚和冬奥冠军谷爱凌的故事，说明了只有做时间的朋友，才能拥有有意义的人生。接下来，作者分析了技术进步给人们带来的便利，但也指出了技术带来的问题，即让人们逐渐模糊了时间的概念。最后，作者提出了自己的建议，即我们应该善用科技，但不要让科技无孔不入地包围我们的生活。

在表达方面，作者的语言流畅，逻辑清晰，观点明确。作者运用了丰富的例子和引用，使得文章更加有说服力和感染力。同时，作者也注意到了文章的节奏和韵律，使得文章读起来更加舒适。

在发展等级方面，作者不仅停留在对现象的简单描述和观点的陈述上，还通过举例、对比等手法，深入分析了技术发展对时间利用的影响，并提出了自己的见解和建议。这体现了作者较高的思维能力和分析能力。

在扣分项方面，作者没有出现偏离题意、观点模糊、内容空洞、语言不通顺等问题。整篇文章紧扣题目要求，观点明确，内容丰富，语言通顺。

在残篇评定方面，由于作文内容完整，没有出现残篇现象，因此不需要进行残篇评定。

综上所述，这篇作文在内容、表达、发展等级等方面都表现得比较出色，没有明显的扣分项，因此我给出的评分是55分。


可以看到模型的输出仅是根据其自己的思路，而并不能满足我们的要求，这主要原因在于prompt中并没有给出足够详细的指导，此时我们可以再次通过 APO 基于我们的示例输出来优化我们的 prompt。

传入的示例输入输出以 Dataset 格式表示，可以传入多组数据

In [ ]:
from qianfan.dataset import Dataset

sample = Dataset.create_from_pyobj(
    [target], 
    input_columns=['question', 'article'],
    reference_column='expect'
)

apo_prompt = prompt.apo_by_sample(sample)

In [34]:
print(apo_prompt.template)



请按照以下要求对高考作文进行综合评价，并提供具体的改进建议。

**评价要求**：

1. **内容与主题契合度**：评估作文是否紧密围绕{question}展开，是否明确回答了题目所提出的问题或触及了题目所探讨的主题。特别注意作文内容是否充实、立意是否新颖，以及是否能够引发深入思考。

2. **语言表达与结构**：分析作文的语言是否流畅、准确，有无语法错误或拼写错误。同时，评估作文的结构是否清晰，段落安排是否合理，以及过渡是否自然。注意检查作文中的论证是否充分，逻辑是否严谨。

3. **深度与创意**：评价作文在思想深度、见解独到性和创新性方面的表现。考察作文是否能够提出深刻的观点或见解，并展现出作者的独特思考和个性。

**改进建议**：

针对作文中存在的不足之处，请提出具体、可操作的改进建议。这些建议可以包括但不限于以下几个方面：内容拓展、语言表达优化、结构调整、论证加强等。

**作文内容**：

{article}

在给出评价时，请尽量提供具体的例子或证据来支持你的观点，并确保评价客观、公正、具有建设性。最后，请给出一个综合评分（满分60分），并简要总结作文的整体表现。




经过优化后，prompt 对于评分要求给出了更为详细的指导，接下来我们可以再来看看模型表现如何

In [31]:
new_r = comp.do(prompt=apo_prompt.render(**target)[0])

print(new_r['result'])

[WARNING] [04-23 12:03:45] base.py:621 [t:140112452616448]: This key `messages` does not seem to be a parameter that the model `ERNIE-3.5-8K` will accept


**综合评价**：

这篇作文紧密围绕题目所给的主题，即技术发展如何影响人们对时间的掌控，进行了深入的思考和讨论。作者从多个角度分析了技术带来的便利与困扰，并提出了自己对时间管理的见解。文章结构清晰，内容丰富，既有理论阐述，也有实例支撑，显示出作者良好的思维能力和表达能力。

**内容与主题契合度**：

作文从人们在新时代技术潮流中如何消磨时间入手，进而引出对时间掌控的探讨。作者明确指出了技术进步带来的便利，如在线教育、即时通讯、快速购物等，同时也指出了其带来的问题，如时间概念的模糊、过度依赖技术等。最后，作者提出了自己对于如何平衡技术利用和时间管理的看法，即“汲取科技的力量，善用时间”。整体来看，作文紧扣主题，立意明确，内容充实，引发了对时间管理的深入思考。

**语言表达与结构**：

作文的语言表达流畅、准确，作者运用了许多生动的比喻和形象的描绘，如“江间波浪兼天涌，塞上风云接地阴”等，增强了文章的艺术感染力。在结构方面，作文采用了总分总的结构，先提出问题，然后分析问题，最后解决问题，逻辑清晰，段落安排合理，过渡自然。

**深度与创意**：

作文在思想深度和见解独到性方面表现良好。作者不仅分析了技术进步对时间管理的影响，还进一步提出了自己对时间管理的看法和建议。这些观点和建议既有理论支撑，又有实例佐证，显示出作者深刻的思考和独特的见解。同时，作文也表现出了一定的创新性，如将技术与时间管理相结合进行讨论，角度新颖，具有一定的启发性。

**改进建议**：

虽然作文在整体上表现良好，但仍有一些地方可以进一步改进。首先，在论证方面，作者可以进一步加强对技术双刃剑特性的分析，提供更多具体的案例或数据来支撑自己的观点。其次，在语言表达方面，作者可以注意避免一些重复的表述，如“我们可以拥抱技术，用时间工具更好地为我们服务”与“汲取科技的力量，善用时间”等，以提高文章的精炼度。最后，在结构方面，作者可以考虑在文章开头或结尾部分加入一些总结性的语句，以更好地概括全文的主旨和观点。

**综合评分**：55分（满分60分）

这篇作文在内容、结构、语言表达等方面都表现出色，对题目所探讨的主题进行了深入的思考和讨论。虽然在一些细节方面还有改进的空间，但整体上是一篇优秀的作文。


In [21]:
from IPython.display import display, Markdown

output = """
|Prompt|Output|
|------|------|
|{}|{}|
|{}|{}|
""".format(
    prompt.template.replace("\n", "<br>"),
    r["result"].replace("\n", "<br>"),
    apo_prompt.template.replace("\n", "<br>"),
    new_r["result"].replace("\n", "<br>"),
)

display(Markdown(output))


|Prompt|Output|
|------|------|
|请帮我批改如下高考作文，从内容、表达、发展等级、扣分项和残篇评定几个角度给出评价和评分（0-60分）：<br>题目：{question}<br>作文内容：{article}|这篇作文是一篇论述技术发展对时间利用影响的文章，作者从个人经历和社会现象出发，深入探讨了技术发展对时间掌控的影响，以及如何在技术的影响下更好地利用时间。<br><br>在内容方面，作者首先描述了人们在新时代技术潮流中如何消磨时间，接着提出了自己的观点，即我们应该做时间的主人，去欣赏生活的美好。然后，作者通过举例航天员翟志刚和冬奥冠军谷爱凌的故事，说明了只有做时间的朋友，才能拥有有意义的人生。接下来，作者分析了技术进步给人们带来的便利，但也指出了技术带来的问题，即让人们逐渐模糊了时间的概念。最后，作者提出了自己的建议，即我们应该善用科技，但不要让科技无孔不入地包围我们的生活。<br><br>在表达方面，作者的语言流畅，逻辑清晰，观点明确。作者运用了丰富的例子和引用，使得文章更加有说服力和感染力。同时，作者也注意到了文章的节奏和韵律，使得文章读起来更加舒适。<br><br>在发展等级方面，作者不仅停留在对现象的简单描述和观点的陈述上，还通过举例、对比等手法，深入分析了技术发展对时间利用的影响，并提出了自己的见解和建议。这体现了作者较高的思维能力和分析能力。<br><br>在扣分项方面，作者没有出现偏离题意、观点模糊、内容空洞、语言不通顺等问题。整篇文章紧扣题目要求，观点明确，内容丰富，语言通顺。<br><br>在残篇评定方面，由于作文内容完整，没有出现残篇现象，因此不需要进行残篇评定。<br><br>综上所述，这篇作文在内容、表达、发展等级等方面都表现得比较出色，没有明显的扣分项，因此我给出的评分是55分。|
|请按照以下要求对高考作文进行综合评价，并提供具体的改进建议。<br><br>**评价要求**：<br><br>1. **内容与主题契合度**：评估作文是否紧密围绕{question}展开，是否明确回答了题目所提出的问题或触及了题目所探讨的主题。特别注意作文内容是否充实、立意是否新颖，以及是否能够引发深入思考。<br><br>2. **语言表达与结构**：分析作文的语言是否流畅、准确，有无语法错误或拼写错误。同时，评估作文的结构是否清晰，段落安排是否合理，以及过渡是否自然。注意检查作文中的论证是否充分，逻辑是否严谨。<br><br>3. **深度与创意**：评价作文在思想深度、见解独到性和创新性方面的表现。考察作文是否能够提出深刻的观点或见解，并展现出作者的独特思考和个性。<br><br>**改进建议**：<br><br>针对作文中存在的不足之处，请提出具体、可操作的改进建议。这些建议可以包括但不限于以下几个方面：内容拓展、语言表达优化、结构调整、论证加强等。<br><br>**作文内容**：<br><br>{article}<br><br>在给出评价时，请尽量提供具体的例子或证据来支持你的观点，并确保评价客观、公正、具有建设性。最后，请给出一个综合评分（满分60分），并简要总结作文的整体表现。<br>|**综合评价**：<br><br>这篇作文紧密围绕题目所给的主题，即技术发展如何影响人们对时间的掌控，进行了深入的思考和讨论。作者从多个角度分析了技术带来的便利与困扰，并提出了自己对时间管理的见解。文章结构清晰，内容丰富，既有理论阐述，也有实例支撑，显示出作者良好的思维能力和表达能力。<br><br>**内容与主题契合度**：<br><br>作文从人们在新时代技术潮流中如何消磨时间入手，进而引出对时间掌控的探讨。作者明确指出了技术进步带来的便利，如在线教育、即时通讯、快速购物等，同时也指出了其带来的问题，如时间概念的模糊、过度依赖技术等。最后，作者提出了自己对于如何平衡技术利用和时间管理的看法，即“汲取科技的力量，善用时间”。整体来看，作文紧扣主题，立意明确，内容充实，引发了对时间管理的深入思考。<br><br>**语言表达与结构**：<br><br>作文的语言表达流畅、准确，作者运用了许多生动的比喻和形象的描绘，如“江间波浪兼天涌，塞上风云接地阴”等，增强了文章的艺术感染力。在结构方面，作文采用了总分总的结构，先提出问题，然后分析问题，最后解决问题，逻辑清晰，段落安排合理，过渡自然。<br><br>**深度与创意**：<br><br>作文在思想深度和见解独到性方面表现良好。作者不仅分析了技术进步对时间管理的影响，还进一步提出了自己对时间管理的看法和建议。这些观点和建议既有理论支撑，又有实例佐证，显示出作者深刻的思考和独特的见解。同时，作文也表现出了一定的创新性，如将技术与时间管理相结合进行讨论，角度新颖，具有一定的启发性。<br><br>**改进建议**：<br><br>虽然作文在整体上表现良好，但仍有一些地方可以进一步改进。首先，在论证方面，作者可以进一步加强对技术双刃剑特性的分析，提供更多具体的案例或数据来支撑自己的观点。其次，在语言表达方面，作者可以注意避免一些重复的表述，如“我们可以拥抱技术，用时间工具更好地为我们服务”与“汲取科技的力量，善用时间”等，以提高文章的精炼度。最后，在结构方面，作者可以考虑在文章开头或结尾部分加入一些总结性的语句，以更好地概括全文的主旨和观点。<br><br>**综合评分**：55分（满分60分）<br><br>这篇作文在内容、结构、语言表达等方面都表现出色，对题目所探讨的主题进行了深入的思考和讨论。虽然在一些细节方面还有改进的空间，但整体上是一篇优秀的作文。<br>|


从输出可以看到，此时模型的输出更加具体地分析了作文的内容、语言和结构，并提出了针对性的批改建议，结构也更加清晰明了。

由此可见，经过 APO 优化后的 prompt 能够帮助我们更好地完成任务，同时避免具有难度的编写 prompt 环节。